<a href="https://colab.research.google.com/github/francescocaforio/CriptoLab/blob/main/CriptoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crittografia simmetrica con Data Encryption Standard: teoria e pratica in Python

Il notebook è stato sviluppato nell’ambito del laboratorio *CriptoLab – Alla scoperta della crittografia*, realizzato presso l’IISS Majorana di Martina Franca e finanziato con risorse PNRR - DM 65/2023.
Analizza e implementa l’algoritmo DES, uno degli algoritmi di crittografia simmetrica più importanti nella storia dell’informatica. Si presenta come un laboratorio didattico in cui teoria e pratica si intrecciano armonicamente: da un lato vengono illustrate le fasi fondamentali del DES, dall’altro si offre la possibilità di osservare e sperimentare il funzionamento del cifrario tramite codice Python eseguibile.
Questo approccio hands-on si dimostra particolarmente efficace per gli studenti, poiché consente di comprendere i meccanismi complessi della cifratura a blocchi attraverso esempi concreti.

A cura di *Francesco Paolo Caforio*, docente classe di concorso A-041 (Scienze e Tecnologie Informatiche) - IISS Majorana, Martina Franca, TA

# Studio strutturato e implementazione semplificata dell'algoritmo crittografico

Il codice definisce alcune tabelle fondamentali per il funzionamento dell’algoritmo DES.
La lista `initial_permutation` rappresenta la permutazione iniziale applicata al blocco di 64 bit in input: essa riorganizza i bit secondo un ordine prestabilito, preparando i dati per i 16 round dell’algoritmo.
Analogamente, la lista `final_permutation` esegue la permutazione finale inversa al termine della cifratura, restituendo il blocco cifrato finale.
La `expansion_table `serve invece a espandere i 32 bit della metà destra del blocco (R) a 48 bit, duplicando alcuni bit, per poterli combinare con la sottochiave del round tramite un'operazione XOR.
Infine, le otto tabelle `S1...S8` rappresentano le S-box: strutture fondamentali che realizzano la sostituzione non lineare nel DES. Ogni S-box prende in input 6 bit e restituisce 4 bit, introducendo confusione nel testo cifrato e rendendo il cifrario resistente ad attacchi crittanalitici.

In [55]:
initial_permutation = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

final_permutation = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

expansion_table = [
    32, 1, 2, 3, 4, 5,
     4, 5, 6, 7, 8, 9,
     8, 9,10,11,12,13,
    12,13,14,15,16,17,
    16,17,18,19,20,21,
    20,21,22,23,24,25,
    24,25,26,27,28,29,
    28,29,31,31,32,1
]

# Definizione delle 8 S-box (dalle Tabelle 6.3–6.10)
S1 = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 10, 3, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
]

S2 = [
    [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
    [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
    [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
    [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
]

S3 = [
    [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
    [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
    [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
    [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
]

S4 = [
    [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
    [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
    [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
    [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
]

S5 = [
    [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
    [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
    [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
    [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
]

S6 = [
    [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
    [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
    [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
    [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
]

S7 = [
    [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
    [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
    [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
    [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
]

S8 = [
    [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
    [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
    [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
    [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
]

Questo codice Python genera casualmente una stringa di 64 bit.

In [40]:
import random

B4 = ""
for _ in range(64):
    B4 += random.choice("01")

print(B4)

0011010000111001001000001000011100110010000001000011001011111111


Il codice definisce la funzione `apply_permutation`, che applica una permutazione a una stringa binaria di 64 bit secondo una tabella di permutazione (`initial_permutation`). La funzione controlla prima che la stringa in input  sia lunga esattamente 64 bit, altrimenti solleva un errore.
Se la lunghezza è corretta, costruisce e restituisce una nuova stringa in cui i bit vengono riordinati secondo i valori indicati nella tabella di permutazione.
Nel caso specifico, `apply_permutation(BL4, initial_permutation)` applica la permutazione iniziale dell’algoritmo DES al blocco binario casuale `BL4`, ottenendo il risultato `ip_result`, che viene poi stampato a schermo. Questa operazione è il primo passo del processo di cifratura DES, e prepara i dati per i successivi round di elaborazione.

In [56]:
def apply_permutation(bitstring, permutation_table):
    if len(bitstring) != 64:
        raise ValueError("La stringa deve contenere esattamente 64 bit.")
    result = ""
    for i in permutation_table:
        result += bitstring[i - 1]
    return result

ip_result = apply_permutation(B4, initial_permutation)
print("Initial Permutation:", ip_result)

Initial Permutation: 1000000011010011101010011000101010001000110101111000001011011000


Il codice definisce e utilizza la funzione `split_64bit_string`, che ha lo scopo di dividere una stringa binaria di 64 bit in due metà da 32 bit. La funzione controlla che la stringa sia lunga esattamente 64 bit, altrimenti solleva un errore. Se la condizione è soddisfatta, estrae la parte sinistra (`left`) e la parte destra (`right`) della stringa, che rappresentano rispettivamente L₀ e R₀, le due metà iniziali del blocco da elaborare nei successivi 16 round del DES. L’assegnazione `R0 = right` rende esplicito che la metà destra sarà usata come base per la catena di trasformazioni.

In [57]:
def split_64bit_string(bitstring):
    """Divide una stringa di 64 bit in due sottostringhe da 32 bit."""
    if len(bitstring) != 64:
        raise ValueError("La stringa deve contenere esattamente 64 bit.")
    left = bitstring[:32]
    right = bitstring[32:]
    return left, right

left, right = split_64bit_string(ip_result)

R0=right

print("Left  (L0):", left)
print("Right (R0):", right)

Left  (L0): 10000000110100111010100110001010
Right (R0): 10001000110101111000001011011000


Il codice genera casualmente una sottochiave binaria di 48 bit.

In [58]:
import random

sottochiave = ''.join(random.choice('01') for _ in range(48))
print(sottochiave)

000011000111001101001100111101011001100000101011


Questo codice implementa una versione semplificata, ma didatticamente significativa del ciclo di 16 round dell’algoritmo DES. Le funzioni `expand_32_to_48`, `xor_48bit` e `apply_sbox` simulano le principali trasformazioni del cifrario. In ogni round, la metà destra del blocco (R0) viene espansa da 32 a 48 bit tramite la `expansion table`, dopodiché subisce un'operazione di XOR con una sottochiave a 48 bit. Il risultato viene diviso in otto blocchi da 6 bit, ciascuno dei quali viene passato a una diversa S-box (S1–S8). Le S-box trasformano ciascun blocco da 6 bit in un blocco da 4 bit, introducendo non linearità nel processo. I risultati delle S-box vengono concatenati in un’unica stringa binaria da 32 bit (`s_final`), che rappresenta l’output della funzione *F* di DES. A questo punto, si calcola l’XOR tra l’output delle S-box e la metà sinistra del blocco (L₀), ottenendo la nuova metà destra (R₁). Infine, L₀ e R₀ vengono aggiornati per il round successivo. L'intero ciclo viene ripetuto per 16 round, come previsto dal DES, anche se in questo caso viene utilizzata la stessa sottochiave fissa per tutti i round (semplificazione utile a scopo didattico). Durante ogni round, vengono stampate le informazioni intermedie per permettere agli studenti di osservare e comprendere nel dettaglio il funzionamento interno dell’algoritmo.

In [60]:
def expand_32_to_48(bitstring32):

    if len(bitstring32) != 32:
        raise ValueError("La stringa deve contenere esattamente 32 bit.")

    result = ""
    for i in expansion_table:
        result += bitstring32[i - 1]

    return result


def xor_48bit(str1, str2):

    if len(str1) != 48 or len(str2) != 48:
        raise ValueError("Entrambe le stringhe devono essere lunghe esattamente 48 bit.")

    result = ""
    for b1, b2 in zip(str1, str2):
        if b1 == b2:
            result += '0'
        else:
            result += '1'

    return result


def xor_32bit(str1, str2):

    if len(str1) != 32 or len(str2) != 32:
        raise ValueError("Entrambe le stringhe devono essere lunghe esattamente 32 bit.")

    result = ""
    for b1, b2 in zip(str1, str2):
        if b1 == b2:
            result += '0'
        else:
            result += '1'

    return result

def apply_sbox(sbox, bits):

    riga = int(bits[0] + bits[5], 2)

    colonna = int(bits[1:5], 2)

    valore = sbox[riga][colonna]
    return format(valore, '04b')

L0=left
R0=right

for i in range(16):

  print("ROUND: ", i)

  expanded = expand_32_to_48(R0)
  print("Espansa a 48 bit: ", expanded)
  #print("Lunghezza: ", len(expanded))

  risultato_xor = xor_48bit(sottochiave, expanded)
  print("XOR: ", risultato_xor)

  if len(risultato_xor) != 48:
    raise ValueError("La stringa deve contenere esattamente 48 bit.")

  b1, b2, b3, b4, b5, b6, b7, b8 = [risultato_xor[i:i+6] for i in range(0, 48, 6)]

  #print("b1:", b1)
  #print("b2:", b2)
  #print("b3:", b3)
  #print("b4:", b4)
  #print("b5:", b5)
  #print("b6:", b6)
  #print("b7:", b7)
  #print("b8:", b8)

  blocks = [b1, b2, b3, b4, b5, b6, b7, b8]

  s1 = apply_sbox(S1, b1)
  s2 = apply_sbox(S2, b2)
  s3 = apply_sbox(S3, b3)
  s4 = apply_sbox(S4, b4)
  s5 = apply_sbox(S5, b5)
  s6 = apply_sbox(S6, b6)
  s7 = apply_sbox(S7, b7)
  s8 = apply_sbox(S8, b8)

  print(f"s1: {s1}")
  print(f"s2: {s2}")
  print(f"s3: {s3}")
  print(f"s4: {s4}")
  print(f"s5: {s5}")
  print(f"s6: {s6}")
  print(f"s7: {s7}")
  print(f"s8: {s8}")

  s_final = s1 + s2 + s3 + s4 + s5 + s6 + s7 + s8
  print(f"s_final: {s_final}")

  if len(s_final) != 32:
    raise ValueError("La stringa deve contenere esattamente 32 bit.")

  risultato_xor = xor_32bit(L0, s_final)
  print("XOR: ", risultato_xor)

  L0=R0
  R0=risultato_xor

ROUND:  0
Espansa a 48 bit:  010001010001011010101111110000000101011011110001
XOR:  010010010110010111100011001101011100111011011010
s1: 1010
s2: 1101
s3: 1110
s4: 1111
s5: 1101
s6: 0101
s7: 0010
s8: 0000
s_final: 10101101111011111101010100100000
XOR:  00101101001111000111110010101010
ROUND:  1
Espansa a 48 bit:  000101011010100111111000001111111001010101011100
XOR:  000110011101101010110100110010100000110101110111
s1: 0001
s2: 1011
s3: 1111
s4: 0011
s5: 1001
s6: 1001
s7: 0000
s8: 0000
s_final: 00011011111100111001100100000000
XOR:  10010011001001000001101111011000
ROUND:  2
Espansa a 48 bit:  010010100110100100001000000011110111111011110001
XOR:  010001100001101001000100111110101110011011011010
s1: 0011
s2: 1101
s3: 0110
s4: 1110
s5: 1110
s6: 0011
s7: 1111
s8: 0000
s_final: 00111101011011101110001111110000
XOR:  00010000010100101001111101011010
ROUND:  3
Espansa a 48 bit:  000010100000001010100101010011111110101011111100
XOR:  000001100111000111101001101110100111001011010111
s1: 0000


Questa parte finale del codice completa il processo di cifratura simulato con il DES. Dopo i 16 round, le due metà finali del blocco (L0 e R0) vengono concatenate in ordine (L0‖R0) e memorizzate nella variabile f. Su questa stringa di 64 bit viene poi applicata la permutazione finale (`final_permutation`), che riorganizza i bit secondo una tabella fissa, ottenendo il risultato fp_result. Successivamente, `fp_result` viene suddiviso nuovamente in due metà da 32 bit (`left` e `right`) con la funzione `split_64bit_string`. Infine, le due metà vengono scambiate (R‖L invece di L‖R) e concatenate nella variabile scambio. Questo scambio finale riflette il comportamento standard dell’algoritmo DES, che dopo i 16 round restituisce il blocco finale cifrato come R₁‖L₁.

In [63]:
f = L0 + R0

fp_result = apply_permutation(f, final_permutation)
print("Final Permutation: ", fp_result)

left, right = split_64bit_string(fp_result)
#print(left)
#print(right)

scambio = right+left

print("Result: ", scambio)

Final Permutation:  1011111011011010010100110000100001010110111101110000111110011110
Result:  0101011011110111000011111001111010111110110110100101001100001000
